In [1]:
import sys
sys.path.append("../")

import tables
import numpy as np
import h5py
from pathlib import Path
import time

from kdmc.utils import compute_sample_energy

In [2]:
data_path = Path("/mnt/Data/Datasets/synthetic/signal/")
filename = "srml2018_4.mat"

In [ ]:
st = time.process_time()

with h5py.File(data_path.joinpath(filename), 'r', rdcc_nbytes=1024**2*4000, rdcc_w0=1) as f:
    snr = np.squeeze(f['snrs'][:])
    rx_x = f['rx_x'][:]
    rx_x = np.swapaxes(rx_x, 0, 2)
    rx_x = rx_x.astype(np.float32)
    # Normalize rx_x
    rx_x = rx_x / compute_sample_energy(rx_x)[:, np.newaxis, np.newaxis]
    rx_s = f['rx_s'][:]
    rx_s = np.swapaxes(rx_s, 0, 2)
    y = f['y'][:]
    y = np.swapaxes(y, 0, 1)
    y = y.astype(np.float32)

    modulation = np.argmax(y, axis=1)
    modulation = modulation.astype(np.int64)

    sps = np.squeeze(f['lsps'][:])
    rolloff = np.squeeze(f['rolloffs'][:])
    fs = np.squeeze(f['fss'][:])
    channel = np.squeeze(f['rays'][:])
    snrs_filt = np.squeeze(f['snrs_filt'][:])
    
et = time.process_time()
print('CPU Execution time:', et - st, 'seconds')

In [ ]:
with open(data_path.joinpath('rx_x.npy'), 'wb') as f:
    np.save(f, rx_x)
with open(data_path.joinpath('rx_s.npy'), 'wb') as f:
    np.save(f, rx_s)
np.savez(data_path.joinpath('srml2018/params.npz'), y=y, snr=snr, snr_filt=snr_filt, sps=sps, rolloff=rolloff, fs=fs, channel=channel)

In [4]:
st = time.process_time()

params = np.load(data_path.joinpath('srml2018/params.npz'))
y = params['y']
snr = params['snr']
snr_filt = params['snr_filt']
sps = params['sps']
rolloff = params['rolloff']
fs = params['fs']
channel = params['channel']
rx_x = np.load(data_path.joinpath('srml2018/rx_x.npy'), mmap_mode='r')
rx_s = np.load(data_path.joinpath('srml2018/rx_s.npy'), mmap_mode='r')
    
et = time.process_time()
print('CPU Execution time:', et - st, 'seconds')

CPU Execution time: 0.13565239400000006 seconds
